In [64]:
### Generate RMSE position and velocity for clustering

# Answering the question which orbits are the best:

import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from ipynb.fs.full.funcCR3BP import CR3BP
from ipynb.fs.full.funcDataGen import *
from ipynb.fs.full.funcPert import *
from ipynb.fs.full.EKFfunc import *
from ipynb.fs.full.funcRMSE import *

In [65]:
# Massive Data Frame:
# Load Observer Orbit
df1 = pd.read_excel("Earth-Moon System/L1_Halo_Northern_Orbits.xlsx")
df2 = pd.read_excel("Earth-Moon System/L1_Halo_Southern_Orbits.xlsx")
df3 = pd.read_excel("Earth-Moon System/L1_Axial.xlsx")
df4 = pd.read_excel("Earth-Moon System/L2_Axial.xlsx")
df5 = pd.read_excel("Earth-Moon System/L1_Lypunov.xlsx")
df6 = pd.read_excel("Earth-Moon System/L2_Lypunov.xlsx")
df7 = pd.read_excel("Earth-Moon System/L2_Halo_Northern_Orbits.xlsx")
df8 = pd.read_excel("Earth-Moon System/L2_Halo_Southern_Orbits.xlsx")
df9 = pd.read_excel("Earth-Moon System/L1_Vertical.xlsx")
df10 = pd.read_excel("Earth-Moon System/L2_Vertical.xlsx")

L1_HN, L1_HS = df1.to_numpy(), df2.to_numpy() # L1 Halo
L2_HN, L2_HS = df7.to_numpy(), df8.to_numpy() # L2 Halo
L1_A, L2_A = df3.to_numpy(), df4.to_numpy() # L1, L2 Axial
L1_L, L2_L = df5.to_numpy(), df6.to_numpy() # L1, L2 Lypunov
L1_V, L2_V = df9.to_numpy(), df10.to_numpy() # L1, L2 Vertical

In [66]:
# Constants of CR3BP
mu = 1.215058560962404E-2
DU = 389703
TU = 382981

In [67]:
# Parameters for random sampling (# Units of km and km/s)
# Inital Conditions
# 
r = 1737.1
rpert = 35000
vpert = 0.20
N = 100 
Tdes = 36*3600/TU
tsteps = 6*3600/TU

# Perturbation
xpert = np.array([[1/DU], [-1/DU], [0]])
vpertIC = np.array([[0.01*(TU/DU)], [-0.01*(TU/DU)], [0]])

Pi = np.zeros((6,6))

for i in range(3):
    Pi[i,i] += 1/DU
    Pi[i+3,i+3] += 0.001 * (TU/DU)

In [68]:
# Massive Observer Matrix
# This cell generates the observer orbits database use for performance calculation
nt = 10 # This get change depending on the amount of observers we want to sample
# For full observer sample utilize the full size of the data matrix

O_IC1 = LHC_Sample(nt, L1_HN[:,7].min(), L1_HN[:,7].max(), L1_HN) #1
O_IC2 = LHC_Sample(nt, L1_HS[:,7].min(), L1_HS[:,7].max(), L1_HS) #2
O_IC3 = LHC_Sample(nt, L2_HN[:,7].min(), L2_HN[:,7].max(), L2_HN) #3
O_IC4 = LHC_Sample(nt, L2_HS[:,7].min(), L2_HS[:,7].max(), L2_HS) #4
O_IC5 = LHC_Sample(nt, L1_A[:,7].min(), L1_A[:,7].max(), L1_A) #5
O_IC6 = LHC_Sample(nt, L2_A[:,7].min(), L2_A[:,7].max(), L2_A) #6
O_IC7 = LHC_Sample(nt, L1_L[:,7].min(), L1_L[:,7].max(), L1_L) #7
O_IC8 = LHC_Sample(nt, L2_L[:,7].min(), L2_L[:,7].max(), L2_L) #8
O_IC9 = LHC_Sample(nt, L1_V[:,7].min(), L1_V[:,7].max(), L1_V) #9
O_IC10 = LHC_Sample(nt, L2_V[:,7].min(), L2_V[:,7].max(), L2_V) #10

# # Mark corresponding data with a number to idnetify orbit
F1 = np.zeros(np.size(O_IC1,0)) + 1
F2 = np.zeros(np.size(O_IC2,0)) + 2
F3 = np.zeros(np.size(O_IC3,0)) + 3
F4 = np.zeros(np.size(O_IC4,0)) + 4
F5 = np.zeros(np.size(O_IC5,0)) + 5
F6 = np.zeros(np.size(O_IC6,0)) + 6
F7 = np.zeros(np.size(O_IC7,0)) + 7
F8 = np.zeros(np.size(O_IC8,0)) + 8
F9 = np.zeros(np.size(O_IC9,0)) + 9
F10 = np.zeros(np.size(O_IC10,0)) + 10

Features = np.concatenate((F1,F2, F3, F4, F5, F6, F7, F8, F9, F10))

XIC_data = np.concatenate((O_IC1[:,1:7],O_IC2[:,1:7], O_IC3[:,1:7], O_IC4[:,1:7], O_IC5[:,1:7], O_IC6[:,1:7], O_IC7[:,1:7], O_IC8[:,1:7], O_IC9[:,1:7], O_IC10[:,1:7]))

dataO = np.concatenate((XIC_data, Features.reshape(-1,1)), axis = 1)

XT = randomSampleIC(r, rpert, vpert, DU, TU, mu, N)

In [69]:
idx1 = np.size(XIC_data, 0)
idx2 = np.size(XT, 1)
avgRMSEpos = np.zeros(idx1)
avgRMSEvel = np.zeros(idx1)
Epos, Evel = np.zeros(idx2), np.zeros(idx2)

for i in range(idx1):
    
    for j in range(idx2):
        
        xT_p = XICpertb(XT[:, j], np.array([-0.001, 0.001]), np.array([-0.001, 0.001]))  # Pertb inital condtions by a bit
        
        DO, DT = XIC_data[i, :], XT[:, j] # Construct DATA ONE TIME STEP
        
        # Updating one inital state
        X, P = AngleEKF(DT[0:3], DO[0:3], xT_p.reshape(-1,1), Pi, 192.0118*(np.pi/(180*3600)) )
        
        # Calculate Performance Factor
        Epos[j], Evel[j] = magRMSE(DT.reshape(-1,1), X)
        
    avgRMSEpos[i], avgRMSEvel[i] = np.mean(Epos), np.mean(Evel)

# avgRMSEpos and avgRMSEvel will be transported into a excel folder or data array folder

In [70]:
### This file will be run to generate performance dataset for Classification Problem
### Inital Results show that there is little relationship between orbit and performance
### Chaotic Nature of the behavior indicate that tracking is a case by case basis